# ***Classical***

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.metrics import accuracy_score  # For calculating accuracy

# Generate valid and invalid boards for training
def generate_n_queens_boards(n, num_samples):
    boards = []
    labels = []
    for _ in range(num_samples):
        board = np.zeros((n, n))
        row_positions = random.sample(range(n), n)  # Randomly place queens on each row
        for i in range(n):
            board[i][row_positions[i]] = 1
        boards.append(board)
        if is_valid_solution(board, n):
            labels.append(1)  # Valid solution
        else:
            labels.append(0)  # Invalid solution
    return np.array(boards), np.array(labels)

# Check if the board is a valid N-Queens solution
def is_valid_solution(board, n):
    rows, cols = np.where(board == 1)
    for i in range(len(rows)):
        for j in range(i + 1, len(rows)):
            if abs(rows[i] - rows[j]) == abs(cols[i] - cols[j]):
                return False  # Queens are attacking diagonally
    return True

# Neural network for classification
class NQueensClassifier(nn.Module):
    def __init__(self, n):
        super(NQueensClassifier, self).__init__()
        self.fc1 = nn.Linear(n * n, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

# Training parameters
n = 8  # N-Queens problem size
num_samples = 1000
learning_rate = 0.001
epochs = 20

# Generate training data
boards, labels = generate_n_queens_boards(n, num_samples)
boards = boards.reshape(num_samples, -1)  # Flatten the boards

# Split data into training and test sets
train_boards, test_boards, train_labels, test_labels = train_test_split(boards, labels, test_size=0.2, random_state=42)

# Convert to torch tensors
train_data = torch.FloatTensor(train_boards)
train_labels = torch.FloatTensor(train_labels).unsqueeze(1)
test_data = torch.FloatTensor(test_boards)
test_labels = torch.FloatTensor(test_labels).unsqueeze(1)

# Initialize the model, optimizer, and loss function
model = NQueensClassifier(n)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(train_data)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test data
with torch.no_grad():
    test_outputs = model(test_data)
    test_preds = (test_outputs > 0.5).float()  # Apply threshold of 0.5 for classification
    accuracy = accuracy_score(test_labels, test_preds)
    print(f'Accuracy on test set: {accuracy * 100:.2f}%')

print("Training complete!")


Epoch [5/20], Loss: 0.5594
Epoch [10/20], Loss: 0.4537
Epoch [15/20], Loss: 0.3277
Epoch [20/20], Loss: 0.1985
Accuracy on test set: 99.50%
Training complete!


**Performance montoring**

In [ ]:
import time

# Classical N-Queens solution using backtracking
def is_safe(board, row, col, N):
    for i in range(col):
        if board[row][i] == 1:
            return False
    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    for i, j in zip(range(row, N, 1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    return True

def solve_n_queens(board, col, N):
    if col >= N:
        return True
    for i in range(N):
        if is_safe(board, i, col, N):
            board[i][col] = 1
            if solve_n_queens(board, col + 1, N):
                return True
            board[i][col] = 0
    return False

def classical_n_queens(N):
    board = [[0 for _ in range(N)] for _ in range(N)]
    if solve_n_queens(board, 0, N):
        return board
    else:
        return None

def benchmark_classical(N):
    start_time = time.time()
    solution = classical_n_queens(N)
    end_time = time.time()
    exec_time = end_time - start_time
    print(f"Classical solution for N={N}:")
    if solution:
        for row in solution:
            print(row)
    else:
        print("No solution found")
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time

N = 16
classical_time = benchmark_classical(N)


Classical solution for N=16:
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Execution time: 0.6269 seconds



# ***Quantum***

In [ ]:
!pip install pennylane pennylane-qiskit qiskit

In [ ]:
import pennylane as qml
from pennylane import qaoa
import numpy as np
import time

# Define the cost Hamiltonian for N-Queens (penalizing attacks)
def cost_hamiltonian(N):
    coeffs = []
    obs = []
    for i in range(N):
        for j in range(i + 1, N):
            coeffs.append(1.0)  # Penalty for queens attacking each other
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Add interaction terms for conflicts
    return qml.Hamiltonian(coeffs, obs)

# Define the mixer Hamiltonian for N-Queens
def mixer_hamiltonian(N):
    coeffs = [-1.0] * N  # Coefficients for the X terms
    obs = [qml.PauliX(i) for i in range(N)]
    return qml.Hamiltonian(coeffs, obs)

# QAOA circuit for N-Queens
def qaoa_n_queens(N, p=1):
    dev = qml.device("default.qubit", wires=N, shots=100)  # Define quantum device with 100 shots

    cost_h = cost_hamiltonian(N)
    mixer_h = mixer_hamiltonian(N)

    # Define the QAOA circuit
    def qaoa_circuit(params):
        qaoa.cost_layer(params[0], cost_h)
        qaoa.mixer_layer(params[1], mixer_h)
        return [qml.sample(qml.PauliZ(i)) for i in range(N)]  # Return samples

    # QNode using the circuit
    qnode = qml.QNode(qaoa_circuit, dev)

    # Random initialization of parameters
    params = [np.random.uniform(0, 2 * np.pi, p), np.random.uniform(0, 2 * np.pi, p)]

    # Gradient descent optimizer
    opt = qml.GradientDescentOptimizer(stepsize=0.1)

    # Optimization loop
    for i in range(20):
        params, _ = opt.step_and_cost(qnode, params)  # Remove the cost display

    # Return the optimized circuit
    return qnode(params)

# Function to check if the sampled board is a valid N-Queens solution
def is_valid_solution(queen_positions, N):
    for i in range(N):
        for j in range(i + 1, N):
            if abs(i - j) == abs(queen_positions[i] - queen_positions[j]):
                return False  # Queens are attacking diagonally
    return True

# Benchmarking quantum-inspired QAOA approach
def benchmark_quantum(N):
    start_time = time.time()
    samples = qaoa_n_queens(N)  # Get the samples from the QAOA circuit
    end_time = time.time()

    # Convert samples from {+1, -1} to {0, 1} (binary)
    solutions = ((np.array(samples) + 1) / 2).astype(int)

    # Calculate accuracy
    valid_count = 0
    for sample in solutions:
        if is_valid_solution(sample, N):
            valid_count += 1

    accuracy = valid_count / len(solutions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    exec_time = end_time - start_time
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time

# Example for N=8
N = 8
quantum_time = benchmark_quantum(N)

/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add train

Iteration 1: Cost = [array([ 1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
       -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
        1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,
       -1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,
       -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,
       -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
        1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), array([ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
        1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
       -1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,
        1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,
        1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
        1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,

/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add train

Iteration 6: Cost = [array([ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,
       -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
        1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.]), array([ 1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
       -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
        1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,
        1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,
       -1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,

/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add train

Iteration 11: Cost = [array([ 1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,
       -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,
        1.,  1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,
       -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,
        1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
        1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.,
        1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.]), array([-1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,
        1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.,
        1., -1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,
        1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,
        1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,
       -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.

/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:158: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add train

Iteration 16: Cost = [array([-1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,
        1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,
        1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,
       -1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
        1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
       -1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,
        1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.]), array([ 1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,
       -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1., -1.,  1.,
       -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,
        1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1., -1.

**perfomance**

In [ ]:
import pennylane as qml
from pennylane import qaoa
import numpy as np
import time

# Quantum-inspired N-Queens using QAOA
def cost_hamiltonian(N):
    coeffs = []
    obs = []
    for i in range(N):
        for j in range(i + 1, N):
            coeffs.append(1.0)
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))
    return qml.Hamiltonian(coeffs, obs)

def mixer_hamiltonian(N):
    coeffs = [-1.0] * N
    obs = [qml.PauliX(i) for i in range(N)]
    return qml.Hamiltonian(coeffs, obs)

def qaoa_n_queens(N, p=1):
    dev = qml.device("default.qubit", wires=N)
    cost_h = cost_hamiltonian(N)
    mixer_h = mixer_hamiltonian(N)

    @qml.qnode(dev)
    def circuit(params):
        qaoa.cost_layer(params[0], cost_h)
        qaoa.mixer_layer(params[1], mixer_h)
        return qml.expval(cost_h)

    params = np.random.rand(2, p)
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    for _ in range(100):  # 100 optimization steps
        params = opt.step(lambda v: circuit(v), params)

    return circuit(params)

# Benchmarking quantum-inspired QAOA approach
def benchmark_quantum(N):
    start_time = time.time()
    energy = qaoa_n_queens(N)
    end_time = time.time()
    exec_time = end_time - start_time
    print(f"Quantum-inspired QAOA solution for N={N}:")
    print(f"Energy: {energy}")
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time

# Example for N=8
N = 8
quantum_time = benchmark_quantum(N)


**Performance**
**Quantum-inspired QAOA solution for N=8:**

***Energy:*** [23.5969326]

***Execution time:*** 5.0386 seconds

# ***Classical Vs Quantum***

In [ ]:
import pennylane as qml
from pennylane import qaoa
import numpy as np
import time
import matplotlib.pyplot as plt

# Define the cost Hamiltonian for N-Queens (penalizing attacks)
def cost_hamiltonian(N):
    coeffs = []
    obs = []
    for i in range(N):
        for j in range(i + 1, N):
            coeffs.append(1.0)  # Penalty for queens attacking each other
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Add interaction terms for conflicts
    return qml.Hamiltonian(coeffs, obs)

# Define the mixer Hamiltonian for N-Queens
def mixer_hamiltonian(N):
    coeffs = [-1.0] * N  # Coefficients for the X terms
    obs = [qml.PauliX(i) for i in range(N)]
    return qml.Hamiltonian(coeffs, obs)

# QAOA circuit for N-Queens
def qaoa_n_queens(N, p=5):  # Reduce QAOA depth to 5
    dev = qml.device("lightning.qubit", wires=N)  # Use faster lightning.qubit simulator

    cost_h = cost_hamiltonian(N)
    mixer_h = mixer_hamiltonian(N)

    # Define the QAOA circuit
    def qaoa_circuit(params):
        qaoa.cost_layer(params[0], cost_h)
        qaoa.mixer_layer(params[1], mixer_h)
        return qml.expval(cost_h)  # Calculate the expected value of the cost Hamiltonian

    # QNode using the circuit
    qnode = qml.QNode(qaoa_circuit, dev)

    # Random initialization of parameters
    params = [np.random.uniform(0, 2 * np.pi, p), np.random.uniform(0, 2 * np.pi, p)]

    # Gradient descent optimizer
    opt = qml.GradientDescentOptimizer(stepsize=0.1)

    # Optimization loop (reduced to 10 iterations)
    for i in range(10):  # Reduced from 20 iterations to 10
        params, cost = opt.step_and_cost(qnode, params)
        # Uncomment below to track the optimization process
        # print(f"Iteration {i+1}: Cost = {cost}")

    return cost

# Benchmarking quantum-inspired QAOA approach
def benchmark_quantum(N, p=5):  # Adjust p to control QAOA depth
    start_time = time.time()
    energy = qaoa_n_queens(N, p=p)  # Reduced QAOA depth to 5
    end_time = time.time()
    exec_time = end_time - start_time
    print(f"Quantum-inspired QAOA solution for N={N}:")
    print(f"Energy: {energy}")
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time

# Classical Backtracking for N-Queens with early stopping option
def classical_n_queens(N, stop_at_first=True):
    solutions = []
    def solve(queens):
        if len(queens) == N:
            solutions.append(queens)
            if stop_at_first:
                return True  # Stop as soon as a solution is found
            return False
        for i in range(N):
            if i not in queens and all(abs(i - q) != len(queens) - j for j, q in enumerate(queens)):
                if solve(queens + [i]):
                    return True
    start_time = time.time()
    solve([])  # Start with an empty board
    end_time = time.time()
    exec_time = end_time - start_time
    print(f"Classical solution for N={N} found {len(solutions)} solutions.")
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time, len(solutions)

# Benchmarking and determining the winner
Ns = [4, 8, 12, 16,20,22]  # Larger values of N for testing

for N in Ns:
    print(f"Benchmark for N={N}")

    # Classical benchmark (early exit after first solution)
    classical_time, classical_solutions = classical_n_queens(N, stop_at_first=True)

    # Quantum benchmark (reduce QAOA depth and iterations)
    quantum_time = benchmark_quantum(N, p=5)  # Reduced depth to 5

    print(f"For N={N}: Classical time = {classical_time:.4f} s, Quantum time = {quantum_time:.4f} s\n")


# Benchmark Results for N-Queens Problem

## For N=4
- **Classical Solution**:
  - Solutions found: 1
  - Execution time: 0.0000 seconds
- **Quantum-Inspired QAOA Solution**:
  - Energy:
    ```
    [[1.84893025 1.84893025 1.84893025 1.84893025 1.84893025]
     [3.27913213 3.27913213 3.27913213 3.27913213 3.27913213]
     [2.02093456 2.02093456 2.02093456 2.02093456 2.02093456]
     [0.35211029 0.35211029 0.35211029 0.35211029 0.35211029]
     [5.65680484 5.65680484 5.65680484 5.65680484 5.65680484]]
    ```
  - Execution time: 2.4247 seconds

## For N=8
- **Classical Solution**:
  - Solutions found: 1
  - Execution time: 0.0009 seconds
- **Quantum-Inspired QAOA Solution**:
  - Energy:
    ```
    [[26.98677884 26.98677884 26.98677884 26.98677884 26.98677884]
     [ 1.93231819  1.93231819  1.93231819  1.93231819  1.93231819]
     [ 4.76451892  4.76451892  4.76451892  4.76451892  4.76451892]
     [ 8.97424461  8.97424461  8.97424461  8.97424461  8.97424461]
     [26.60305262 26.60305262 26.60305262 26.60305262 26.60305262]]
    ```
  - Execution time: 2.2114 seconds

## For N=12
- **Classical Solution**:
  - Solutions found: 1
  - Execution time: 0.0022 seconds
- **Quantum-Inspired QAOA Solution**:
  - Energy:
    ```
    [[ 0.39165922  0.39165922  0.39165922  0.39165922  0.39165922]
     [22.83340133 22.83340133 22.83340133 22.83340133 22.83340133]
     [44.68657382 44.68657382 44.68657382 44.68657382 44.68657382]
     [11.36306682 11.36306682 11.36306682 11.36306682 11.36306682]
     [42.07329305 42.07329305 42.07329305 42.07329305 42.07329305]]
    ```
  - Execution time: 3.0066 seconds

## For N=16
- **Classical Solution**:
  - Solutions found: 1
  - Execution time: 0.1165 seconds
- **Quantum-Inspired QAOA Solution**:
  - Energy:
    ```
    [[66.37379896 66.37379896 66.37379896 66.37379896 66.37379896]
     [64.59163521 64.59163521 64.59163521 64.59163521 64.59163521]
     [26.62032521 26.62032521 26.62032521 26.62032521 26.62032521]
     [29.17241735 29.17241735 29.17241735 29.17241735 29.17241735]
     [18.6122225  18.6122225  18.6122225  18.6122225  18.6122225 ]]
    ```
  - Execution time: 22.3476 seconds

## For N=20
- **Classical Solution**:
  - Solutions found: 1
  - Execution time: 3.3730 seconds
- **Quantum-Inspired QAOA Solution**:
  - Energy:
    ```
    [[186.92139792 186.92139792 186.92139792 186.92139792 186.92139792]
     [125.86940242 125.86940242 125.86940242 125.86940242 125.86940242]
     [ 88.00530834  88.00530834  88.00530834  88.00530834  88.00530834]
     [ 19.24940108  19.24940108  19.24940108  19.24940108  19.24940108]
     [  8.59432439   8.59432439   8.59432439   8.59432439   8.59432439]]
    ```
  - Execution time: 646.9327 seconds

## For N=22
- **Classical Solution**:
  - Solutions found: 1
  - Execution time: 33.8719 seconds
- **Quantum-Inspired QAOA Solution**:
  - Energy:
    ```
    [[230.97156958 230.97156958 230.97156958 230.97156958 230.97156958]
     [  7.16856901   7.16856901   7.16856901   7.16856901   7.16856901]
     [165.04990776 165.04990776 165.04990776 165.04990776 165.04990776]
     [ 10.12653912  10.12653912  10.12653912  10.12653912  10.12653912]
     [ 21.53258708  21.53258708  21.53258708  21.53258708  21.53258708]]
    ```
  - Execution time: 3141.0221 seconds


**graph**

In [ ]:
import pennylane as qml
from pennylane import qaoa
import numpy as np
import time
import matplotlib.pyplot as plt

# Define the cost Hamiltonian for N-Queens (penalizing attacks)
def cost_hamiltonian(N):
    coeffs = []
    obs = []
    for i in range(N):
        for j in range(i + 1, N):
            coeffs.append(1.0)  # Penalty for queens attacking each other
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Add interaction terms for conflicts
    return qml.Hamiltonian(coeffs, obs)

# Define the mixer Hamiltonian for N-Queens
def mixer_hamiltonian(N):
    coeffs = [-1.0] * N  # Coefficients for the X terms
    obs = [qml.PauliX(i) for i in range(N)]
    return qml.Hamiltonian(coeffs, obs)

# QAOA circuit for N-Queens
def qaoa_n_queens(N, p=5):  # Reduce QAOA depth to 5
    dev = qml.device("lightning.qubit", wires=N)  # Use faster lightning.qubit simulator

    cost_h = cost_hamiltonian(N)
    mixer_h = mixer_hamiltonian(N)

    # Define the QAOA circuit
    def qaoa_circuit(params):
        qaoa.cost_layer(params[0], cost_h)
        qaoa.mixer_layer(params[1], mixer_h)
        return qml.expval(cost_h)  # Calculate the expected value of the cost Hamiltonian

    # QNode using the circuit
    qnode = qml.QNode(qaoa_circuit, dev)

    # Random initialization of parameters
    params = [np.random.uniform(0, 2 * np.pi, p), np.random.uniform(0, 2 * np.pi, p)]

    # Gradient descent optimizer
    opt = qml.GradientDescentOptimizer(stepsize=0.1)

    # Optimization loop (reduced to 10 iterations)
    for i in range(10):  # Reduced from 20 iterations to 10
        params, cost = opt.step_and_cost(qnode, params)
        # Uncomment below to track the optimization process
        # print(f"Iteration {i+1}: Cost = {cost}")

    return cost

# Benchmarking quantum-inspired QAOA approach
def benchmark_quantum(N, p=5):  # Adjust p to control QAOA depth
    start_time = time.time()
    energy = qaoa_n_queens(N, p=p)  # Reduced QAOA depth to 5
    end_time = time.time()
    exec_time = end_time - start_time
    print(f"Quantum-inspired QAOA solution for N={N}:")
    print(f"Energy: {energy}")
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time

# Classical Backtracking for N-Queens with early stopping option
def classical_n_queens(N, stop_at_first=True):
    solutions = []
    def solve(queens):
        if len(queens) == N:
            solutions.append(queens)
            if stop_at_first:
                return True  # Stop as soon as a solution is found
            return False
        for i in range(N):
            if i not in queens and all(abs(i - q) != len(queens) - j for j, q in enumerate(queens)):
                if solve(queens + [i]):
                    return True
    start_time = time.time()
    solve([])  # Start with an empty board
    end_time = time.time()
    exec_time = end_time - start_time
    print(f"Classical solution for N={N} found {len(solutions)} solutions.")
    print(f"Execution time: {exec_time:.4f} seconds\n")
    return exec_time, len(solutions)

# Benchmarking and determining the winner
Ns = [4, 8, 12, 16, 20, 22]  # Larger values of N for testing

classical_times = []
quantum_times = []

for N in Ns:
    print(f"Benchmark for N={N}")

    # Classical benchmark (early exit after first solution)
    classical_time, classical_solutions = classical_n_queens(N, stop_at_first=True)
    classical_times.append(classical_time)

    # Quantum benchmark (reduce QAOA depth and iterations)
    quantum_time = benchmark_quantum(N, p=5)  # Reduced depth to 5
    quantum_times.append(quantum_time)

    print(f"For N={N}: Classical time = {classical_time:.4f} s, Quantum time = {quantum_time:.4f} s\n")

# Plot the execution times
plt.figure(figsize=(10, 6))
plt.plot(Ns, classical_times, label="Classical Time", marker='o')
plt.plot(Ns, quantum_times, label="Quantum Time", marker='o')
plt.yscale("log")  # Logarithmic scale for better visibility
plt.xlabel("Number of Queens (N)")
plt.ylabel("Execution Time (seconds)")
plt.title("Classical vs Quantum Execution Time for N-Queens Problem")
plt.legend()
plt.grid(True)
plt.show()


# N-Queens Benchmark Results

| N  | Classical Solution | Classical Execution Time (s) | Quantum Solution | Quantum Execution Time (s) |
|----|---------------------|-----------------------------|------------------|----------------------------|
| 4  | 1                  | 0.0001                      | QAOA Solution    | 2.0302                     |
| 8  | 1                  | 0.0009                      | QAOA Solution    | 1.3077                     |
| 12 | 1                  | 0.0027                      | QAOA Solution    | 3.2144                     |
| 16 | 1                  | 0.1221                      | QAOA Solution    | 21.5952                    |
| 20 | 1                  | 3.1303                      | QAOA Solution    | N/A (warning during execution) |

### Visualization of Solution
![Comparision of Quantum and Classical model speeds](https://drive.google.com/uc?export=view&id=1zqCGIj4GsrlCd-cgPinDpzainy0X0jTC)


